In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as sts
from scipy.special import gamma
import matplotlib.pyplot as plt

In [2]:
data = np.load('lda.npz')
data_train = data['train']
data_test = data['test']

data_train = data_train[data_train[:,0]<200]
data_test = data_train[data_train[:,0]<50]

data_train[:,0] = data_train[:,0]-1
data_test[:,0] = data_test[:,0]-1
data_train[:,1] = data_train[:,1]-1
data_test[:,1] = data_test[:,1]-1

voc = max(data_train[:,1])+1
doc = max(data_train[:,0])+1
k = 10
print (voc,doc)

12419 199


In [3]:
class PFA():
    def __init__(self,voc,doc,data_train,data_test,eta=0.5,c=1,c0=1,r0=1,gamma=1,alpha=0.05,eps=0.05,k=10):
        #initialization
        #hyperparameter
        self.eta = 0.5
        self.c = 1
        self.c0 = 1
        self.r0 = 1
        self.gamma = 1
        self.alpha = 0.05
        self.eps = 0.05
        
        #number of document,word type,topic
        self.topic_num = 10 #主题数量
        self.voc_num = voc #词汇数
        self.doc_num = doc #文档数
        
        #gengerative process sample
        self.phi = sts.dirichlet.rvs([self.alpha]*self.voc,size = self.k)  #Topic-word matrix  (voc * k matrix)
        self.pk = sts.beta.rvs(self.c*self.eps,self.c*(1-self.eps),size =self.k) # pk 的分布 1*k matrix
        self.rk = sts.gamma.rvs(self.c0*self.r0,scale=1/self.c0,size=self.k) #rk 1*k matrix
        self.ki = np.array([sts.gamma.rvs(self.rk,scale = self.pk/(1-self.pk)) for i in range(self.doc)]) # doc * k matrix
        
        #sampling count
        self.xik = np.zeros((self.doc,self.k)) #第i个主题第k个topic
        self.xpk = np.zeros((self.voc,self.k)) #第k个主题第p个词的数量
        self.xk = np.zeros((self.k))           #第k个topic的数量
        self.xi = np.zeros((self.doc))         #第i个文档的数量
        
        self.per_plot = []
        
        self.data_test = data_test
        self.data_train = data_train
        
    def sampler(self):
        #1.Count assignment
        for doc_word_count in self.data_train:
            
            doc_index,word_index,word_count = doc_word_count #[0,7,2]
            
            prob =   self.phi[:,word_index] * self.ki[doc_index] # 单词在每个主题的概率 * 该文档每个主题的概率
            prob/=sum(prob+0.01) 
            res = sts.multinomial.rvs(word_count,p=prob)
                
            self.xik[doc_index] += res   #第i个文档第k个topic的数量
            self.xpk[word_index] += res          #第k个主题第p个词的数量
            self.xk += res                       #第k个topic的数量
            self.xi[doc_index] += word_count     #第i个文档的数量
        
        #更新topic-doc  matrix,就是我们要替换的部分
        for topic_index in range(self.k):
            self.phi[topic_index] = sts.dirichlet.rvs([self.alpha]*self.voc + self.xpk[:,topic_index])
            
        #更新pk    
        for topic_index in range(self.k):   
            self.pk[topic_index] = sts.beta.rvs(self.c * self.eps + self.xk[topic_index] ,self.c *(1- self.eps)+ self.doc *  self.rk[topic_index] )
        
        #sample rk
        for topic_index in range(self.k):    
            if self.xk[topic_index] == 0: #当xk=0,负二项分布退化为伯努利分布
                self.rk[topic_index] = sts.gamma.rvs(self.c0 * self.r0 , scale = 1/(self.c0 - self.doc * np.log(1- self.pk[topic_index])))
            else:
                self.rk[topic_index] = sts.gamma.rvs(self.c0 * self.r0 + np.sum(self.CRT(topic_index)),scale = 1/(self.c0 - self.doc * np.log(1- self.pk[topic_index])))
        
        # sample self.ki
        for topic_index in range(self.k): 
            self.ki = sts.gamma(self.xik + self.rk,self.pk).rvs()
     
    def compute_perplexity(self):
        phi_theta = np.array([np.sum(self.phi * self.ki[doc_index].reshape(-1,1),0) for doc_index in range(self.doc)])
        phi_theta = phi_theta/np.sum(phi_theta,1).reshape(-1,1)            
        mat = np.zeros((self.doc,self.voc)) 
        for index in self.data_test:#Convert sparse to normal matrix
            mat[index[0],index[1]] = index[-1]    
        per = np.sum(mat * np.log(phi_theta))
        res = np.exp(- per/np.sum(self.data_test[:,-1]))
        print (res)
        self.per_plot.append(res)
        return res
    
    def CRT(self,topic): #CRT 
        res = np.array([sum([sts.bernoulli.rvs(self.pk[topic]/(i-1+self.pk[topic])) for i in np.linspace(1,self.xik[d][topic],int(self.xik[d][topic]))]) for d in range(self.doc)])
        return res
    
    
    def demo(self,iteration):
        for it in range(iteration):  
            print ('Begin {}th iterations'.format(it))
            self.sampler()
            self.compute_perplexity()
            
        sns.set()
        plt.plot(self.per_plot)
        plt.xlabel('Iteration')
        plt.ylabel('Perplexity')
        plt.show()
        return self.perplot
            

In [7]:
#model = PFA(voc=voc,doc=doc,data_train=data_train,data_test = data_test,k=k)
#model.demo(20)
mm = PFA(voc=voc,doc=doc,data_train=data_train,data_test = data_test,k=k)
mm.sampler()
xvk = mm.xpk

In [80]:
class Dir_BN():
    def __init__(self,xvk,doc):
        self.doc = doc
        self.voc ,self.k = xvk.shape
        
        #Hyperparameter
        self.a0 = 1
        self.b0 = 1
        self.g0 = 1
        self.h0 = 1
        self.e0 = 0.01
        self.f0 = 0.01
        
        self.eta = 0.05
        self.t =3
        
        #Basic latent variable
        self.psi = np.zeros((self.t,self.voc,self.k))
        self.phi = np.zeros((self.t,self.voc,self.k))
        self.beta = 0.5 * np.ones((self.t-1,self.k,self.k))
        self.beta_gammak = 0.1 * np.ones((self.t-1,self.k))
        self.beta_c = 0.1 * np.ones((self.t-1))
        self.beta_gamma0 = 0.1 * np.ones((self.t-1))
        self.beta_c0 = 0.1 * np.ones((self.t-1))
        
        #Propagate variable
        self.yvk = np.zeros((self.t,self.voc,self.k))
        self.xvk = np.zeros((self.t,self.voc,self.k))
        self.xvk[0] = xvk
        
        for layer in range(self.t):
            self.psi[layer] = self.eta * np.ones((self.voc,self.k))
            self.phi[layer] = sts.gamma.rvs(self.psi[layer])
            self.phi[layer] /= np.sum(self.phi[layer],0)
    
    def propa_sample(self):
        #propagate
        for layer in range(self.t):
            if layer < self.t-1:
                for v in range(self.voc):
                    for k in range(self.k):
                        self.yvk[layer][v][k] = self.CRT(self.xvk[layer][v][k],self.psi[layer][v][k])
                self.xvk[layer+1] = self.latent_count(self.yvk,self.phi,self.beta,layer)         
        
        ## top down
        for layer in np.arange(self.t-1,-1,-1):
            layer = int(layer)
            
            # update psi
            if layer == self.t-1:
                psi = self.sample_eta(self.xvk[layer],self.psi[layer])
                self.psi[layer] = psi * np.ones((self.voc,self.k))
            else:
                self.psi[layer] = np.array([np.sum(self.phi[layer+1] * model.beta[layer][k,:],1) for  k in range(self.k)]).T
        
            #update beta_parameter
            if layer >0:
                qk =  sts.beta.rvs(np.sum(self.psi[layer],0),np.sum(model.xvk[layer],0))
                
            
            
            
            # update phi
            self.phi[layer] = np.array([sts.dirichlet((self.psi[layer] + self.xvk[layer])[:,k_index]).rvs().reshape(-1,) for k_index in range(self.k)]).T
            
        
    def CRT(self,x,y):
        res = 0
        if x >0:
            if x==1:
                res = 1
            else:
                for n in range(int(x)):
                    res+= (sts.uniform.rvs()< y/(n-1+y))
        return res
    
    def latent_count(self,yvk,phi,beta,layer):
        rr = np.zeros((self.voc,self.k))
        for i in range(self.k):
            data = yvk[layer][:,i]
            pval = self.phi[layer] * self.beta[layer][i,:]
            pval = pval/np.sum(pval,1).reshape(-1,1)
            res = np.array([sts.multinomial.rvs(a,b) for a,b in zip(data,pval)])
            rr += res
        return rr
    
    def sample_eta(self,xvk,psi):
        log_p = sum(np.log(sts.beta.rvs(np.sum(psi,0),np.sum(xvk,0))))     
        svk = 0
        for v in range(self.voc):
            for k in range(self.k):
                svk+=self.CRT(xvk[v][k],psi[v][k])
        eta = sts.gamma.rvs(0.1+ svk,1)/(10-self.voc*log_p)
        return eta

In [81]:
model = Dir_BN(xvk)

In [82]:
model.propa_sample()

ValueError: All parameters must be greater than 0

In [99]:
np.sum(model.psi[-1],0)

array([970.03401685, 970.03401685, 970.03401685, 970.03401685,
       970.03401685, 970.03401685, 970.03401685, 970.03401685,
       970.03401685, 970.03401685])

(12419, 10)

In [103]:
xvk.shape

(12419, 10)